# Charlottesville Residential Real Estate Assessment Analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
import string
from folium import plugins
%matplotlib inline
from tqdm import tqdm

import json
from datetime import datetime
import plotly.express as px

sns.set()
plt.rcParams["figure.figsize"] = (20,7)

In [2]:
ass = pd.read_csv('csv/AllAssessments.csv')
ass['Address']=ass['StreetNumber'] +' '+ass['StreetName']
del ass['StreetNumber']
del ass['StreetName']
coors = pd.read_csv('csv/coordinates.csv')
coors.columns=['Address','Latitudue','Longitude']
ass=ass.join(coors.set_index('Address'), on='Address', how='left')
res = pd.read_csv('csv/Residential.csv')
res['Address']=res['StreetNumber'] +' '+res['StreetName']
ass=ass[ass.TaxYear >= 2000]
ass.head()

,RecordID_Int,ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,Unit,Address,Latitudue,Longitude
0,1,010001000,35643600,152423500,188067100,2020,NaN,1117 EMMET ST N,38.054409,-78.499656
1,2,010001000,29172900,149711300,178884200,2019,NaN,1117 EMMET ST N,38.054409,-78.499656
2,3,010001000,25005400,146802400,171807800,2018,NaN,1117 EMMET ST N,38.054409,-78.499656
3,4,010001000,24449500,142363700,166813200,2017,NaN,1117 EMMET ST N,38.054409,-78.499656
4,5,010001000,22848500,100504900,123353400,2016,NaN,1117 EMMET ST N,38.054409,-78.499656


In [3]:
res_ass = ass.merge(res[['Address','UseCode','YearBuilt']], left_on='Address', right_on='Address', how='right').dropna(subset=['TotalValue']).drop_duplicates(keep='first')
neighs = pd.read_csv('csv/neighborhood_sf.csv')
n_ass=res_ass.join(neighs[['Address','Neighborhood']].set_index('Address'), on='Address', how='left').dropna(subset=['Neighborhood'])

In [4]:
n_ass.head()


,RecordID_Int,ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,Unit,Address,Latitudue,Longitude,UseCode,YearBuilt,Neighborhood
441,359.0,010006000,371400.0,928500.0,1299900.0,2020.0,NaN,2028 BARRACKS RD,38.050765,-78.49808,Single Family,1940.0,Venable
442,360.0,010006000,360600.0,884300.0,1244900.0,2019.0,NaN,2028 BARRACKS RD,38.050765,-78.49808,Single Family,1940.0,Venable
443,361.0,010006000,300500.0,808400.0,1108900.0,2018.0,NaN,2028 BARRACKS RD,38.050765,-78.49808,Single Family,1940.0,Venable
444,362.0,010006000,280800.0,769900.0,1050700.0,2017.0,NaN,2028 BARRACKS RD,38.050765,-78.49808,Single Family,1940.0,Venable
445,363.0,010006000,244200.0,756400.0,1000600.0,2016.0,NaN,2028 BARRACKS RD,38.050765,-78.49808,Single Family,1940.0,Venable


### Average Year Built vs. Average 2020 Assessment Value by Neighborhood

In [5]:
n_ass[n_ass.TaxYear == 2000].groupby(['Neighborhood'])[['YearBuilt', 'TotalValue']].mean()


,YearBuilt,TotalValue
Neighborhood,,
10th & Page,1945.041096,46104.452055
Barracks / Rugby,1957.229958,193432.328482
Belmont,1948.279520,68024.933511
Fifeville,1946.068182,59724.848485
Fry's Spring,1952.113260,96399.631676
Greenbrier,1960.582468,165747.866205
Jefferson Park Avenue,1940.230000,122570.000000
Johnson Village,1963.788945,117118.592965
Lewis Mountain,1931.596154,229513.942308


In [14]:
fig=px.scatter(n_ass.loc[n_ass['TaxYear'] == 2000].groupby('Neighborhood')[['YearBuilt', 'TotalValue']].mean().drop('Rose Hill').reset_index(), x='YearBuilt', y='TotalValue', hover_name='Neighborhood',color='Neighborhood',text='Neighborhood')
fig.update_traces(textposition='top center')
fig.update_layout(
    height=500,
    width=800,
    title_text='Avg. 2020 Total Assessment Value vs. Avg. Year Built'
)
fig.write_html("html/assessvsyear.html")

### Percent Change in Real Estate Assessment Value by Neighborhood

In [7]:
u_ass=n_ass.copy(deep=True)
u_ass['Unit'] = u_ass['Unit'].fillna('')
u_ass['Address'] = u_ass['Address'] + ' ' + u_ass['Unit']
u_ass=u_ass.drop(['Unit'], axis=1)
u_ass

,RecordID_Int,ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear,Address,Latitudue,Longitude,UseCode,YearBuilt,Neighborhood
441,359.0,010006000,371400.0,928500.0,1299900.0,2020.0,2028 BARRACKS RD,38.050765,-78.498080,Single Family,1940.0,Venable
442,360.0,010006000,360600.0,884300.0,1244900.0,2019.0,2028 BARRACKS RD,38.050765,-78.498080,Single Family,1940.0,Venable
443,361.0,010006000,300500.0,808400.0,1108900.0,2018.0,2028 BARRACKS RD,38.050765,-78.498080,Single Family,1940.0,Venable
444,362.0,010006000,280800.0,769900.0,1050700.0,2017.0,2028 BARRACKS RD,38.050765,-78.498080,Single Family,1940.0,Venable
445,363.0,010006000,244200.0,756400.0,1000600.0,2016.0,2028 BARRACKS RD,38.050765,-78.498080,Single Family,1940.0,Venable
...,...,...,...,...,...,...,...,...,...,...,...,...
1755552,344469.0,610104000,16000.0,77400.0,93400.0,2002.0,1806 EASTVIEW ST,38.014544,-78.472197,Single Family,1946.0,Belmont
1755553,344470.0,610104000,13200.0,63900.0,77100.0,2001.0,1806 EASTVIEW ST,38.014544,-78.472197,Single Family,1946.0,Belmont
1755553,344470.0,610104000,13200.0,63900.0,77100.0,2001.0,1806 EASTVIEW ST,38.014544,-78.472197,Single Family,1946.0,Belmont
1755554,344471.0,610104000,13200.0,63900.0,77100.0,2000.0,1806 EASTVIEW ST,38.014544,-78.472197,Single Family,1946.0,Belmont


In [8]:
s_ass=u_ass[['Address','Neighborhood','UseCode','TaxYear', 'TotalValue']].sort_values(['Address', 'TaxYear'])

s_ass=s_ass.drop_duplicates(subset=['Address','Neighborhood','UseCode','TaxYear'])
s_ass=s_ass.drop_duplicates()
s_ass=s_ass.loc[s_ass.TotalValue != 0]
s_ass['PctChange']=s_ass.groupby('Address', sort=False)['TotalValue'].apply(lambda x: x.pct_change()).to_numpy()*100
s_ass

,Address,Neighborhood,UseCode,TaxYear,TotalValue,PctChange
12852,1 LATROBE CT,Venable,Single Family,2000.0,42000.0,NaN
12851,1 LATROBE CT,Venable,Single Family,2001.0,44100.0,5.000000
12850,1 LATROBE CT,Venable,Single Family,2002.0,44700.0,1.360544
12849,1 LATROBE CT,Venable,Single Family,2003.0,48000.0,7.382550
12848,1 LATROBE CT,Venable,Single Family,2004.0,55200.0,15.000000
...,...,...,...,...,...,...
1665899,981 RIVES ST,Belmont,Single Family,2016.0,138400.0,0.000000
1665898,981 RIVES ST,Belmont,Single Family,2017.0,148500.0,7.297688
1665897,981 RIVES ST,Belmont,Single Family,2018.0,163300.0,9.966330
1665896,981 RIVES ST,Belmont,Single Family,2019.0,176400.0,8.022045


In [9]:
grouped = s_ass[['Neighborhood', 'TaxYear', 'PctChange']].groupby(['Neighborhood', 'TaxYear']).mean()
unstacked=s_ass[['Neighborhood', 'TaxYear', 'PctChange']].groupby(['Neighborhood', 'TaxYear']).mean().unstack()
avg=unstacked.drop(unstacked.columns[0], axis=1).mean(axis=1).reset_index()
avg5 = unstacked.iloc[:,15:21].mean(axis=1).reset_index()
avg5.rename(columns={0:"Avg Pct Change 2015-2020"}, inplace=True)

avg.rename(columns={0:"Avg Pct Change 2000-2020"}, inplace=True)
unstacked=unstacked.transpose().reset_index().drop(['level_0'], axis=1)
unstacked

Neighborhood,TaxYear,10th & Page,Barracks / Rugby,Belmont,Fifeville,Fry's Spring,Greenbrier,Jefferson Park Avenue,Johnson Village,Lewis Mountain,Locust Grove,Martha Jefferson,North Downtown,Ridge Street,Rose Hill,Starr Hill,The Meadows,Venable,Woolen Mills
0,2000.0,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001.0,12.330926,13.264466,10.768891,22.772125,17.986265,11.670610,9.429263,10.612508,10.371980,12.539350,16.987219,12.214510,17.577612,5.337295,5.543787,4.632012,9.468975,20.137996
2,2002.0,6.461777,21.725294,14.047665,2.750167,8.989455,13.547884,12.286358,8.408851,10.725593,11.687036,7.335425,12.527233,9.080111,11.625124,1.820999,16.204516,11.717351,38.735454
3,2003.0,6.564581,9.932647,26.521113,17.699675,14.870534,11.283701,14.934453,11.928126,18.375682,18.743434,18.728074,13.580663,19.018642,37.414202,75.912697,5.631566,18.698994,27.550982
4,2004.0,23.854166,26.124662,22.021259,26.956372,15.501984,12.332569,16.033769,8.539018,14.843972,17.464097,24.472109,18.153420,14.668232,35.661843,17.320598,13.448818,19.670111,43.442870
5,2005.0,33.937581,16.145506,20.737868,30.152632,18.470342,11.760455,29.596694,14.624653,9.667187,19.761015,15.584971,32.941516,108.522473,14.737430,26.873748,12.216227,27.293340,21.464790
6,2006.0,146.474927,29.250460,23.388065,32.674264,22.469303,13.160700,27.580340,19.063875,19.169825,40.620571,27.531611,17.057519,54.220332,82.026701,16.160407,27.821313,53.297336,20.191353
7,2007.0,65.454366,13.004855,26.405608,39.076292,32.394879,19.777241,-0.136114,87.130240,6.140192,22.815708,8.454524,13.959029,31.709260,11.122732,23.570938,7.809667,20.387914,37.931701
8,2008.0,10.778968,0.623103,17.404318,8.042071,17.015322,0.601440,5.696487,168.559573,7.375769,12.701440,8.041718,2.235699,70.896029,1.433992,8.870133,7.217783,2.556962,7.789060
9,2009.0,13.227847,0.969103,3.581260,9.527977,-0.303942,-3.445151,0.846067,75.476091,4.491833,-0.037777,3.187117,-1.848181,38.121399,0.489095,5.746589,-5.789377,5.781879,5.449337


# Plotting Data

In [10]:
avg20yr=px.bar(avg, x='Neighborhood',y='Avg Pct Change 2000-2020',color='Neighborhood')
avg20yr.write_html("html/avg20yr_ass.html")

In [11]:
avg20yr

In [12]:
avg5yr=px.bar(avg5, x='Neighborhood',y='Avg Pct Change 2015-2020',color='Neighborhood')
avg5yr.write_html("html/avg5yr_ass.html")

In [13]:
avg5yr

In [264]:
import plotly.graph_objects as go

In [301]:
fig = go.Figure()

for column in unstacked.columns[1:]:
    fig.add_trace(go.Scatter(x=unstacked['TaxYear'],
                             y=unstacked[column],
                             mode='lines',
                             name=column))
fig.update_xaxes(title_text='Tax Year')
fig.update_yaxes(title_text='Average Percent Change in Total Assessment Value')

fig.show()
fig.write_html("html/allneighs.html")